# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# TD9 : Régression bayésienne

La régression ridge est un cas particulier de la régression linéaire bayésienne lorsque la loi *a priori* partiellement informative suivante est utilisée :
$$ f_{(\boldsymbol{\beta},\sigma^2)}(\boldsymbol{\beta},\sigma^2) \propto \mathcal{N}\left(\boldsymbol{\beta} \, \left| \, \boldsymbol{0}_p , \frac{\sigma^2}{\lambda}I_p \right. \right) \times \frac{1}{\sigma^2}. $$
Pour être conséquent avec l'approche bayésienne, il faudrait fixer l'hyperparamètre $\lambda$ avant même d'avoir vu les données. De façon pragmatique, la valeur de $\lambda$ est souvent choisie par validation comme nous le ferons dans cet exercice.

## Données

Le jeu de données étudiées pour l'exercice 1 provient de l'étude de Tibshirani et al. (2004) illustrant les méthodes de régression modernes telle que la régression ridge. Nous étudierons la glycémie sanguine (glu) en fonction de 8 variables explicatives : l'âge du patient (age), l'indice de masse corporel du patient (bmi), de la tension artérielle moyenne (map) ainsi que de 5 autres indicateur sanguins (tc, ldl, hdl, tch et ltg). Nous laissons tomber dans le caadre de ce TD le sexe du patient et la variable prog.


#### Référence
Tibshirani, R. J., Johnstone, I., Hastie, T., & Efron, B. (2004). Least angle regression. *The Annals of Statistics*, 32(2), 407–499. DOI:10.1214/009053604000000067


In [ ]:
using CSV 
using DataFrames
using Distributions
using Gadfly
using LinearAlgebra
import Random
using Statistics
import StatsBase

## Chargement des données

In [ ]:
data = CSV.read("diabetes.csv", DataFrame)
select!(data, Not([:prog, :sex]))
first(data,5)

## Création de l'ensemble d'entraînement et de validation

In [ ]:
Random.seed!(3302)

ntrain = round(Int, .8*nrow(data))

train_id = sample(1:nrow(data), ntrain, replace=false, ordered=true)
valid_id = setdiff(1:nrow(data), train_id)

train = data[train_id, :]
valid = data[valid_id, :]

## Estimation des paramètres de standardisation des variables

Paramètres estimés avec l'ensemble d'entraînement.

In [ ]:
# Standardisation des variables
dt = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(train), dims=1)
transdata = StatsBase.transform(dt, Matrix{Float64}(train))

X = transdata[:,1:8]
y = transdata[:,9]

n = length(y)
m = size(X,2)

# Exercice 1 : Régression Ridge

### (a) Estimez les coefficients de régression Ridge en utilisant $\lambda = 0$.

### (b) Estimez les coefficients de régression Ridge en utilisant $\lambda = 1$.

### (c) Pour $0 \leq \lambda \leq 10$, estimez les coefficients de régression puis affichez-les en fonction de $\lambda$.

Ce graphique s'appelle la trace des coefficients de régression. Lorsqu'il y a multicolinéarité, les estimations des coefficients de régression varient beaucoup en fonction de $\lambda$.

Je vous propose de remplir la matrice $\hat\beta$ constituée de $p$ colonnes. Chaque ligne correspondra aux estimations des coefficients de régression pour une valeur de $\lambda$ donnée. Cette matrice sera par la suite transformée en DataFrame pour un affichage simplifié.

### (d) Estimez λ en minisant l'erreur de prédiction sur l'échantillon de validation

### (e) Tracez la loi *a posteriori* marginale du coefficient de régression correspond à la variable *age*.

Est-ce que cette variable possède un pouvoir prédictif significatif ? 

Utiliser la commande `LocationScale(μ, sqrt(Σ), TDist(ν))` pour créer un objet de type *loi de Student multidimensionnelle à ν degrés de liberté, de paramètre de localisation μ et de paramètre d'échelle Σ*.

### (f) Implémentez l'échantillonnage de Gibbs pour générer un échantillon de la loi *a posteriori* de la régression ridge.

Utilisez votre estimation λ̂.

## Exercice 2 : Exploration stochastique de l'espace des modèles [OPTIONEL]

Dans cet exercice, on reprend le jeux de données *bodyfat.csv* comportant le pourcentage de gras de 20 patientes en fonction des variables explicatives suivantes :

$x_1$ : l'épaisseur des plis de la peau des triceps (en mm) ;<br/>
$x_2$ : le tour de cuisse (en mm) ;<br/>
$x_3$ : la circonférence du bras en (mm).<br/>

Nous allons implémenter l'échantillonnage de Gibbs pour explorer de façon stochastique l'espace des modèele. Puisqu'il n'y a que 3 variables explicatives dans ce cas-ci, il est possible de déterminer quel est le *vrai* meilleur modèle par énumération complète de tous les modèles.


In [ ]:
# Chargement des données

data = CSV.read("bodyfat.csv", DataFrame)
first(data, 5)

In [ ]:
# Standardisation des variables
dt = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(data), dims=1)
transdata = StatsBase.transform(dt, Matrix{Float64}(data))

X = transdata[:,1:3]
y = transdata[:,4]

n = length(y)
m = size(X,2)

In [ ]:
# Énumration de tous les modèles possibles

itr = reverse.(Iterators.product(fill(0:1,m)...))[:]

M = BitArray[]
for i in eachindex(itr)
   push!(M, [itr[i]...] .== 1)
end
M

### (a) Écrivez une fonction permettant de calculer le BIC d'un modèle.

La fonction devrait prendre comme argument un vecteur de booléens indicant quelles variables sont incluses dans le modèle. La fonction devrait prendre en argument par exemple le vecteur [true true true].

In [ ]:
function getbic(γ::BitArray)
    
    #ToDo
end

### (b) Calculer le BIC pour chacun des modèles possibles

Quel est le meilleur modèle ?

### (c) Exécuter la cellule suivante pour parcourir l'espace des modèles avec l'échantillonnage de Gibbs

In [ ]:
function softmax(x::AbstractVector{<:Real})
    
    z = x .- maximum(x)
    
    e = exp.(z)
    
    s = e ./ sum(e)
    
end

In [ ]:
niter = 10000

γ = trues(3,niter)

for i=2:niter
   γ[:,i] = γ[:,i-1]
    for j=1:m
        
        η₀ = γ[:,i]
        η₁ = γ[:,i]
        η₀[j] = false
        η₁[j] = true
        
#         θ = exp(getbic(η₁)) / ( exp(getbic(η₀)) + exp(getbic(η₁)))
        s = softmax(getbic.([η₀, η₁])) # évite l'instabilité numérique
        θ = s[2]
        
        γ[j,i] = rand(Bernoulli(θ))
        
    end
end

### (d) Exécuter les cellules suivantes pour déterminer le modèle choisi pour chacune des itérations

In [ ]:
function modelid(M::Vector{BitArray},γ::BitArray)
    
    @assert length(γ)==length(M[1])
    
    ind = 1
    
    while M[ind] != γ
        ind +=1
    end
    
    return ind
        
end

In [ ]:
id = [modelid(M,γ[:,i]) for i=1:size(γ,2)]

df = DataFrame(Itération = 1:niter, Modèle = id)

groupby(df, :Modèle)

nb = combine(groupby(df, :Modèle), :Itération=> length => :Itérations)

sort!(nb, :Itérations, rev=true)

### (e) Selon les résultats en (d), quel est le meilleur modèle ? Est-ce cohérent avec le BIC calculé en (a) ?